In [1]:
!pip install pandas matplotlib seaborn scikit-learn


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git clone https://github.com/empt22/ci-cd-project.git

Cloning into 'ci-cd-project'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 52 (delta 9), reused 45 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 5.03 KiB | 5.03 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [6]:
import pandas as pd

In [7]:


data = {'car': ['A', 'B', 'C', 'D'], 'Price': [20000, 21000, 20500, 23000]}

df = pd.DataFrame(data)

print(df)

  car  Price
0   A  20000
1   B  21000
2   C  20500
3   D  23000


In [13]:
%cd /content/ci-cd-project/data_analysis
#!pwd
#!ls

/content/ci-cd-project/data_analysis


In [14]:
!cp /content/Kaggle_Cars_2024_Data_Summary.ipynb /content/ci-cd-project/data_analysis/


cp: cannot stat '/content/Kaggle_Cars_2024_Data_Summary.ipynb': No such file or directory


In [15]:
!ls /content/

ci-cd-project  drive  sample_data


In [ ]:
!ls /content/drive/MyDrive/ColabNotebooks